In [42]:
import pandas as pd

# Align Demo
* Working from the [Biopython tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec385)
* Prerequisites
    * Reading on PSSMs
* Compare human homolog to a different sequence
    * Compute a [log odds substitution matrix](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec390)
    * What are the log odds of the following polymorphisms?
        * Hydrophobic -> hydrophilic and vice versa
        * Aromatic -> non-aromatic and vice versa
    * Construct a more generalized PSSM for the above categories of "penalized polymorphisms"
* Homework: repeat analysis with a different homolog
    * Generate a generalized PSSM for a different type of penalized polymorphism, such as acidic -> basic residues

We're using [Biopython](http://biopython.org/DIST/docs/tutorial/Tutorial.html) again. If the below import commands fail, you might need to install Biopython from the command line:
```bash
pip install biopython

# or using poetry
poetry add biopython
```

In [61]:
from Bio.Align import AlignInfo, MultipleSeqAlignment
from Bio import AlignIO

# Read the alignment records
We use the Python function from the Biopython package: `Bio.AlignIO.read` to read the trimmed alignment file. This Python function reads the `*.txt` file in the `'fasta'` format and returns an instance of `Bio.Align.MultipleSeqAlignment` (documentation can be found [here](https://biopython.org/DIST/docs/api/Bio.Align.MultipleSeqAlignment-class.html) and [here](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec81)).

In [9]:
alignment = AlignIO.read(open('./trimmed_alg.txt'), format='fasta')
alignment

<<class 'Bio.Align.MultipleSeqAlignment'> instance (9757 records of length 60918, SingleLetterAlphabet()) at 7f5306a42450>

Each element of this list-like instance is a sequence:

In [24]:
alignment[0] 

SeqRecord(seq=Seq('------------------------------------------------------...---', SingleLetterAlphabet()), id='7260.FBpp0240705', name='7260.FBpp0240705', description='7260.FBpp0240705', dbxrefs=[])

This instance of `Bio.Align.MultipleSeqAlignment` is a lot like a Python list. For instance, you can:

In [40]:
# get the number of sequences in this alignment
print("number of sequence records: ", len(alignment))

# iterate over the sequence records in the alignment
record_counter = 0
for record in alignment:
    record_counter += 1
print("number of sequence records (a different way): ", sequence_counter)

# get the 100th sequence record in the alignment
print("ID of the 100th sequence: ", alignment[99].id)

number of sequence records:  9757
number of sequence records (a different way):  9757
ID of the 100th sequence:  9796.ENSECAP00000016722


# Filter the sequences in the alignment
For now, we're only interested in "domestic species," or species whose scientific name is in the Python list `domestic_sp_names`:

In [22]:
domestic_sp_names = [
    'Homo sapiens', # human
    'Mus musculus', # mouse
    'Canis lupus familiaris', # dog
    'Felis catus', # cat
    'Bos taurus', # cattle
    'Equus caballus', # horse
    'Gallus gallus' # chicken
]

The sequences in the `Bio.Align.MultipleSeqAlignment` are for **all** the species that EggNOG could find, including worms, polar bears, and other species that we're not interested in.

Let's filter out sequences from species whose names are **not** in the list `domestic_sp_names`. To do this, we will:
1. Get the scientific name for each species, and load it into the `description` attribute of each sequence. This should be familiar from the [descriptive stats demo](../descriptive_stats_demo/eggNOG_alignment_metadata.ipynb).
2. Use a [list comprehension](https://github.com/wilfredinni/python-cheatsheet#list-comprehension) to get the list of sequences for species that we are interested in.
3. The above step will generate a Python list; it will need to be converted to an instance of  `Bio.Align.MultipleSeqAlignment` if we want to use fancy Biopython analysis tools on it.

## Step #1: Get scientific name for each species
This should be familiar from the [descriptive stats demo](../descriptive_stats_demo/eggNOG_alignment_metadata.ipynb).

In [56]:
tmprss2_ext = pd.read_table('../descriptive_stats_demo/extended_members.txt', header=None)
tmprss2_ext.columns = ['id_1', 'id_2', 'species', '', '']
# tmprss2_ext.head()

In [55]:
for record in alignment:
    
    # from visual inspection we know the name format is XXXX.unique_id, so we split on "." and take the last element of the list
    id_code = record.id.split('.')[-1]
    
    # reference the metadata to get the species name
    sp_name = tmprss2_ext[tmprss2_ext['id_1'] == id_code]['species'].values
    
    try:
        sp_name = sp_name.item()
    except ValueError:
        sp_name = None
    
    # assign the species name to the species attribute
    record.description = sp_name

## Step #2: Use a list comprehension to filter to domestic species

In [57]:
dom_aln_list = [record for record in alignment
                if record.description in domestic_sp_names]

732

We see that the length of this filtered list is much shorter:

In [60]:
print("number of records for all species:", len(alignment))
print("number of records for domestic species:", len(dom_aln_list))

number of records for all species: 9757
number of records for domestic species: 732


## Step #3: Convert this list to a new `MultipleSeqAlignment` instance

In [62]:
dom_aln = MultipleSeqAlignment(dom_aln_list)

`dom_aln` has the same data, but is a different type of Python variable:

In [63]:
print("dom_aln_list is type:", type(dom_aln_list))
print("dom_aln is type:", type(dom_aln))

dom_aln_list is type: <class 'list'>
dom_aln is type: <class 'Bio.Align.MultipleSeqAlignment'>


# Get the consensus sequence
Before we start comparing sequences to each other, let's generate the consensus sequence ([Wikipedia](https://en.wikipedia.org/wiki/Consensus_sequence#:~:text=In%20molecular%20biology%20and%20bioinformatics,position%20in%20a%20sequence%20alignment.)), which is the sequence we will be comparing to. In this case, we want to compare to the human sequence, so we will only calculate the consensus sequence for records for which the `description` attribute (in which we stored the species name) is `Homo sapiens`.

In [99]:
human_aln_list = [record for record in dom_aln
                  if record.description == 'Homo sapiens']
human_aln = MultipleSeqAlignment(human_aln_list)

In [100]:
len(human_aln)

118

In [101]:
str(human_aln[0].seq).replace("-", "")

'MSVRFRVGREENAQWWLLYLLVPALFCRAGGSIPIPQKLFGEVTSPLFPKPYPNNFETTTVITVPTGYRVKLVFQQFDLEPSEGCFYDYVKISADKKSLGRFCGQLGSPLGNPPGKKEFMSQGNKMLLTFHTDFSNEENGTIMFYKGFLAYYQAVDLDECASRSKSGEEDPQPQCQHLCHNYVGGYFCSCRPGYELQEDRHSCQAECSSELYTEASGYISSLEYPRSYPPDLRCNYSIRVERGLTLHLKFLEPFDIDDHQQVHCPYDQLQIYANGKNIGEFCGKQRPPDLDTSSNAVDLLFFTDESGDSRGWKLRYTTEIIKCPQPKTLDEFTIIQNLQPQYQFRDYFIATCKQGYQLIEGIQVVAGVVDVSAGRVSWSTSSQTCCTRVYTCTAQGIWKNEQKGEKIPRCLPVCGKPVNPVEQRQRIIGGQKAKMGNFPWQVFTNIHGRGGGALLGDRWILTAAHTLYPKEHEAQSNASLDVFLGHTNVEELMKLGNHPIRRVSVHPDYRQDESYNFEGDIALLELENSVTLGPNLLPICLPDNDTFYDLGLMGYVSGFGVMEEKIAHDLRFVRLPVANPQACENWLRGKNRMDVFSQNMFCAGHPSLKQDACQGDSGGVFAVRDPNTDRWVATGIVSWGIGCSRGYGFYTKVLNYVDWIKKEMEEED'

In [64]:
summary_align = AlignInfo.SummaryInfo(dom_aln)
summary_align

We don't actually need to calculate a consensus sequence here, since we will use the `Homo sapiens` sequence as reference. If we did want to generate a consensus sequence:

In [66]:
consensus = summary_align.dumb_consensus()

KeyboardInterrupt: 

In [69]:
len(consensus)

60918

## Compute PSSM

In [ ]:
my_pssm = summary_align.pos_specific_score_matrix(consensus) # chars_to_ignore=["N"]
my_pssm

In [ ]:
my_pssm[1]

In [ ]:
replace_info = summary_align.replacement_dictionary(["G", "A"])
replace_info.keys()

In [14]:
replace_info[replace_info.keys()[0]]

NameError: name 'replace_info' is not defined

## Compute substitution and log odds matrix

In [15]:
from Bio import SubsMat
my_arm = SubsMat.SeqMat(replace_info)
my_arm

NameError: name 'replace_info' is not defined

In [16]:
my_lom = SubsMat.make_log_odds_matrix(my_arm)
my_lom

NameError: name 'my_arm' is not defined